In [ ]:
import nltk
nltk.data.path.append('./nltk_data/')
from nltk.corpus import brown, stopwords
import string
import numpy as np
import sys

In [ ]:
corpus = brown.words()
stopwordsList = stopwords.words('english')
for i in range(len(string.punctuation)):
    stopwordsList.append(string.punctuation[i])
    #stopwordsList.append((string.punctuation[i] + string.punctuation[i]))
corpus = [word.lower() for word in corpus if word.lower() not in stopwordsList]

In [ ]:
wordsList = []
countList = []
for i in range(len(corpus)):
    if(corpus[i] in wordsList):
        countList[wordsList.index(corpus[i])] = countList[wordsList.index(corpus[i])] + 1
    else:
        wordsList.append(corpus[i])
        countList.append(1)

In [ ]:
finalList = []
for i in range(len(wordsList)):
    finalList.append((countList[i], wordsList[i]))

In [ ]:
finalList.sort(reverse=True)
vocab = finalList[:5000]
context = finalList[:1000]
contextWords = []
for i in range(1000):
    contextWords.append(context[i][1])
totalContextWords = 0
for i in range(1000):
    totalContextWords = totalContextWords + context[i][0]
nfunct = np.zeros((len(vocab), len(context)))

In [ ]:
for i in range(len(vocab)):
    indices = [z for z, x in enumerate(corpus) if x == vocab[i][1]]
    for j in indices:
        if(j > 1 and corpus[(j-2)] in contextWords):
            nfunct[i][contextWords.index(corpus[(j-2)])] = nfunct[i][contextWords.index(corpus[(j-2)])] + 1
        if(j > 0 and corpus[(j-1)] in contextWords):
            nfunct[i][contextWords.index(corpus[(j-1)])] = nfunct[i][contextWords.index(corpus[(j-1)])] + 1
        if(j < (len(corpus)-2) and corpus[(j+2)] in contextWords):
            nfunct[i][contextWords.index(corpus[(j+2)])] = nfunct[i][contextWords.index(corpus[(j+2)])] + 1
        if(j > (len(corpus)-1) and corpus[(j+1)] in contextWords):
            nfunct[i][contextWords.index(corpus[(j+1)])] = nfunct[i][contextWords.index(corpus[(j+1)])] + 1
        

In [ ]:
Cembedding = np.zeros((10, len(vocab), len(context)))
for i in range(len(vocab)):
    for j in range(len(context)):
        numer = (nfunct[i][j] / nfunct[i].sum())
        denom = (context[j][0] / totalContextWords)
        temp = (numer/denom)
        if(denom > numer):
            temp = 0
        else:
            temp = np.log(temp)
        Cembedding[int(j/100)][i][(j%100)] = temp
NewEmbed = np.zeros((len(vocab), 100))
for i in range(len(vocab)):
    for j in range(100):
        for k in range(10):
            NewEmbed[i][j] = NewEmbed[i][j] + Cembedding[k][i][(j)]
Cembedding = NewEmbed

In [ ]:
for i in range(5000):
    print(i,": ",vocab[i])

In [ ]:
#predict single words
word1 = 1427
output = -1
minDist = 10000000
for i in range(5000):
    if(i == word1):
        continue
    dist = np.dot(Cembedding[i], Cembedding[word1])
    dist = dist / np.linalg.norm(Cembedding[i])
    dist = dist / np.linalg.norm(Cembedding[word1])
    dist = 1 - dist
    if(dist < minDist):
        minDist = dist
        output = i
print(vocab[word1][1],":",vocab[output][1])

In [ ]:
imput = principalComponents#Cembedding
target = []
for i in range(len(vocab)):
    target.append(vocab[i][1])
target = np.asarray(target)

In [ ]:
#cluster
import numpy as np
from sklearn.cluster import KMeans
from sklearn import datasets

estimator = KMeans(n_clusters=100)
estimator.fit(imput)

In [ ]:
estimator.labels_

In [ ]:
#output clusters
for i in range(len(estimator.labels_)):
    if(estimator.labels_[i] == 5):
        print(target[i])

In [ ]:
from sklearn.decomposition import PCA

Cembedding = np.zeros((len(vocab), len(context)))
for i in range(len(vocab)):
    for j in range(len(context)):
        numer = (nfunct[i][j] / nfunct[i].sum())
        denom = (context[j][0] / totalContextWords)
        temp = (numer/denom)
        if(denom > numer):
            temp = 0
        else:
            temp = np.log(temp)
        Cembedding[i][(j)] = temp

pca = PCA(n_components=100)
principalComponents = pca.fit_transform(Cembedding)

In [ ]:
target.shape